In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

#print(os.listdir("../input"))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.
C:\Users\fourtek12\.conda\envs\tf-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\fourtek12\.conda\envs\tf-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\fourtek12\.conda\envs\tf-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\fourtek12\.conda\envs\tf-gpu\lib

In [2]:
lines=pd.read_csv("newdata.csv",encoding='utf-8')

In [3]:
lines['source'].value_counts()

KeyError: 'source'

In [ ]:
lines=lines[lines['source']=='ted']

In [4]:
lines.head(20)

,Unnamed: 0,english_sentence,hindi_sentence
0,0,politicians do not have permission to do what needs to be done.,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह करने कि अनुमति नहीं है ."
1,1,"I'd like to tell you about one such child,","मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी,"
2,2,This percentage is even greater than the percentage in India.,यह प्रतिशत भारत में हिन्दुओं प्रतिशत से अधिक है।
3,3,what we really mean is that they're bad at not paying attention.,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
4,4,.The ending portion of these Vedas is called Upanishad.,इन्हीं वेदों का अंतिम भाग उपनिषद कहलाता है।
5,5,"The then Governor of Kashmir resisted transfer , but was finally reduced to subjection with the aid of British .","कश्मीर के तत्कालीन गवर्नर ने इस हस्तांतरण का विरोध किया था , लेकिन अंग्रेजों की सहायता से उनकी आवाज दबा दी गयी ."
6,6,In this lies the circumstances of people before you.,इसमें तुमसे पूर्व गुज़रे हुए लोगों के हालात हैं।
7,7,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
8,8,“”Global Warming“” refer to warming caused in recent decades and probability of its continual presence and its indirect effect on human being.,ग्लोबल वॉर्मिंग से आशय हाल ही के दशकों में हुई वार्मिंग और इसके निरंतर बने रहने के अनुमान और इसके अप्रत्यक्ष रूप से मानव पर पड़ने वाले प्रभाव से है।
9,9,You may want your child to go to a school that is not run by the LEA - a non-maintained special school or an independent school that can meet your child 's needs .,"हो सकता है कि आप चाहते हों कि आप का नऋर्नमेनटेन्ड ह्यबिना किसी समर्थन के हृ विशेष स्कूल , या किसी स्वतंत्र स्कूल में जाए , इजसके पास विशेष शैक्षणिक जऋऋरतों वाले बच्चों के प्रति सहूलियत हों . ."


In [5]:
pd.isnull(lines).sum()

Unnamed: 0          0
english_sentence    2
hindi_sentence      0
dtype: int64

In [6]:
lines=lines[~pd.isnull(lines['english_sentence'])]

In [7]:
lines.drop_duplicates(inplace=True)

C:\Users\fourtek12\.conda\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


* ### Let us pick any 25000 rows from the dataset.

In [14]:
lines=lines.sample(n=30000,random_state=42)
lines.shape

(30000, 3)

In [15]:
# Lowercase all characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.lower())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.lower())

In [16]:
# Remove quotes
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub("'", '', x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub("'", '', x))

In [17]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines['english_sentence']=lines['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [18]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.translate(remove_digits))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.translate(remove_digits))

lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.strip())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.strip())
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))


In [19]:
# Add start and end tokens to target sequences
lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x : 'START_ '+ x + ' _END')

In [20]:
lines.head()

,Unnamed: 0,english_sentence,hindi_sentence
53519,53519,totally he published lectures discourses more than novels plays translation work childrens book and thousands of pages of articles reports speeches compositions letters but he attained reputation and status in lectures discourses and stories which he could not get in other facets,START_ उन्होंने कुल उपन्यास से कुछ अधिक कहानियाँ नाटक अनुवाद बालपुस्तकें तथा हजारों पृष्ठों के लेख सम्पादकीय भाषण भूमिका पत्र आदि की रचना की लेकिन जो यश और प्रतिष्ठा उन्हें उपन्यास और कहानियों से प्राप्त हुई वह अन्य विधाओं से प्राप्त न हो सकी। _END
31597,31597,vegetable fried boiled more than the level is considrerd to be strength,START_ आवश्यकता से अधिक तला भुना शाकाहार ग्रहण करना भी राजसिक माना गया है। _END
107171,107171,copyright on target media inc cincinnati oh,START_ कापीराइट आन टार्गेट मीडिया इंक सिनसिनैटी ओहायो _END
143032,15425,shlaspi root extract blood comes out in urine of patient,START_ श्‍लैस्पी मूल अर्क रोगी के पेशाब में रक्‍त आता है । _END
119329,119329,because now shes picked up the highway,START_ क्योंकि अब उन्होनें हाईवे पकड़ ली है _END


In [21]:
### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in lines['english_sentence']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in lines['hindi_sentence']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [22]:
len(all_eng_words)

33976

In [23]:
len(all_hindi_words)

42529

In [24]:
lines['length_eng_sentence']=lines['english_sentence'].apply(lambda x:len(x.split(" ")))
lines['length_hin_sentence']=lines['hindi_sentence'].apply(lambda x:len(x.split(" ")))

In [25]:
lines.head()

,Unnamed: 0,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
53519,53519,totally he published lectures discourses more than novels plays translation work childrens book and thousands of pages of articles reports speeches compositions letters but he attained reputation and status in lectures discourses and stories which he could not get in other facets,START_ उन्होंने कुल उपन्यास से कुछ अधिक कहानियाँ नाटक अनुवाद बालपुस्तकें तथा हजारों पृष्ठों के लेख सम्पादकीय भाषण भूमिका पत्र आदि की रचना की लेकिन जो यश और प्रतिष्ठा उन्हें उपन्यास और कहानियों से प्राप्त हुई वह अन्य विधाओं से प्राप्त न हो सकी। _END,42,45
31597,31597,vegetable fried boiled more than the level is considrerd to be strength,START_ आवश्यकता से अधिक तला भुना शाकाहार ग्रहण करना भी राजसिक माना गया है। _END,12,15
107171,107171,copyright on target media inc cincinnati oh,START_ कापीराइट आन टार्गेट मीडिया इंक सिनसिनैटी ओहायो _END,7,9
143032,15425,shlaspi root extract blood comes out in urine of patient,START_ श्‍लैस्पी मूल अर्क रोगी के पेशाब में रक्‍त आता है । _END,10,13
119329,119329,because now shes picked up the highway,START_ क्योंकि अब उन्होनें हाईवे पकड़ ली है _END,7,9


In [26]:
lines[lines['length_eng_sentence']>30].shape

(2398, 5)

In [27]:
lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_hin_sentence']<=20]

In [28]:
lines.shape

(19941, 5)

In [29]:
print("maximum length of Hindi Sentence ",max(lines['length_hin_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))

maximum length of Hindi Sentence  20
maximum length of English Sentence  20


In [30]:
max_length_src=max(lines['length_hin_sentence'])
max_length_tar=max(lines['length_eng_sentence'])

In [31]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(33976, 42529)

In [32]:
num_decoder_tokens += 1 #for zero padding


In [33]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [34]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [35]:
lines = shuffle(lines)
lines.head(10)

,Unnamed: 0,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
149375,21768,if patient leads his life in moderation then half the problem gets removed on just like that,START_ रोगी अगर परहेज रखकर अपना जीवनयापन करें तो आधी समस्या यूँ ही दूर हो जाती है । _END,17,19
74441,74441,yet as a representative body of the people it is the trustee of their sovereignty,START_ फिर भी लोगों के प्रतिनिधि निकाय के रूप में विधानमंडल उनकी प्रभुसत्ता का प्रतीक होता है _END,15,18
134782,7175,to keep record of birth and death in the region and informing village panchayat officer,START_ क्षेत्र में जन्म व मृत्यु का रिकॉर्ड रखना व ग्राम पंचायत अधिकारी को सूचित करना । _END,15,18
42455,42455,the ramayana is also an excellent example of the love of brothers,START_ रामायण भातृप्रेम का भी उत्कृष्ट उदाहरण है। _END,12,9
147436,19829,patient gets relief very soon by doing this,START_ ऐसा करने से रोगी को बहुत जल्दी आराम आ जाता है । _END,8,14
157907,30300,every year during spring lovers of buddhism come here,START_ हर वर्ष बसन्त के दौरान बौद्ध धर्मप्रेमी यहाँ आते हैं । _END,9,13
164306,36699,the trains of both sides used to come and stand at the sides,START_ दोनों तरफ की ट्रेनें आकर आरपार खड़ी हो जाती थीं । _END,13,13
140313,12706,bhujangasana is extremely important for all the spinal cord diseases like cervical spondylitis and slip disc,START_ सर्वाइकल स्पोण्डोलाइटिस एवं स्लिप डिस्क आदि समस्त मेरुदंड रोगों के लिए भुजंगासन अति महत्त्वपूर्ण है । _END,16,18
93685,93685,the inspiration follows from example,START_ इस प्रेरणा के लिए मिसाल भी जो मौजूद हैं _END,5,11
137220,9613,sugar is white poison,START_ इसी मिश्रण से प्रात और सायं मंजन करें कुछ दिनों में आराम मिलेगा । _END,4,16


### Split the data into train and test

In [36]:
X, y = lines['english_sentence'], lines['hindi_sentence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((15952,), (3989,))

### Let us save this data

In [37]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')


In [38]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder-Decoder Architecture

In [39]:
latent_dim=300

In [40]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens+1, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [41]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [42]:
#tf.math.log
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')


In [43]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    10193100    input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 300)    12759000    input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS

In [ ]:
#if 'session' in locals() and session is not None:
    #print('Close interactive session')
   # session.close()
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 100

model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)




Epoch 1/100
124/124 [==============================] - 82s 664ms/step - loss: 7.1271 - val_loss: 6.7510
Epoch 2/100
124/124 [==============================] - 76s 614ms/step - loss: 6.4593 - val_loss: 6.5372
Epoch 3/100
124/124 [==============================] - 74s 594ms/step - loss: 6.1395 - val_loss: 6.4327
Epoch 4/100
124/124 [==============================] - 75s 606ms/step - loss: 5.8656 - val_loss: 6.2508
Epoch 5/100
124/124 [==============================] - 75s 601ms/step - loss: 5.6354 - val_loss: 6.2035
Epoch 6/100
124/124 [==============================] - 75s 601ms/step - loss: 5.4346 - val_loss: 6.1602
Epoch 7/100
124/124 [==============================] - 75s 606ms/step - loss: 5.2450 - val_loss: 6.1138
Epoch 8/100
124/124 [==============================] - 75s 609ms/step - loss: 5.0638 - val_loss: 6.0883
Epoch 9/100
124/124 [==============================] - 74s 599ms/step - loss: 4.8937 - val_loss: 6.0501
Epoch 10/100
124/124 [==============================] - 74s 598

124/124 [==============================] - 76s 616ms/step - loss: 0.0902 - val_loss: 8.0114
Epoch 78/100
124/124 [==============================] - 77s 623ms/step - loss: 0.0844 - val_loss: 8.0320
Epoch 79/100
124/124 [==============================] - 76s 616ms/step - loss: 0.0796 - val_loss: 8.0269
Epoch 80/100
124/124 [==============================] - 76s 616ms/step - loss: 0.0753 - val_loss: 8.0680
Epoch 81/100
124/124 [==============================] - 90s 729ms/step - loss: 0.0730 - val_loss: 8.0560
Epoch 82/100
124/124 [==============================] - 76s 614ms/step - loss: 0.0682 - val_loss: 8.0991
Epoch 83/100
124/124 [==============================] - 83s 668ms/step - loss: 0.0651 - val_loss: 8.0729
Epoch 84/100
124/124 [==============================] - 84s 676ms/step - loss: 0.0621 - val_loss: 8.1100
Epoch 85/100
124/124 [==============================] - 78s 626ms/step - loss: 0.0595 - val_loss: 8.1158
Epoch 86/100
124/124 [==============================] - 76s 611ms/st

In [ ]:
model.save_weights('nmt_weights.h5')

In [ ]:
model.load_weights('nmt_weights.h5')

In [ ]:
# Converts the unicode file to ascii
import unicodedata
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')


def preprocess_sentence_english(w):
    w = unicode_to_ascii(w.lower().strip())

    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

    w = w.rstrip().strip()

    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w
  
def preprocess_sentence_hindi(w):
    w = unicode_to_ascii(w.strip())

    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
#     w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

    w = w.rstrip().strip()

    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

In [3]:
def evaluate(sentence):
    #attention_plot = np.zeros((max_length_targ, max_length_inp))

    sentence = preprocess_sentence_english(sentence)

    inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_inp,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

        # storing the attention weights to plot later on
       # attention_weights = tf.reshape(attention_weights, (-1, ))
        #attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.index_word[predicted_id] + ' '

        if targ_lang.index_word[predicted_id] == '<end>':
            return result, sentence

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence

In [4]:
def translate(sentence):
    result, sentence = evaluate(sentence)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

   # attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
   # plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [5]:
translate("This is a Book.")

NameError: name 're' is not defined

In [6]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)


NameError: name 'Model' is not defined

In [7]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [8]:
#print(decoded_sentence)

In [9]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_inputs[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
  #  print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

NameError: name 'encoder_inputs' is not defined

In [10]:
for seq_index in [14077,20122,40035,40064, 40056, 40068, 40090, 40095, 40100, 40119, 40131, 40136, 40150, 40153]:
    input_seq = encoder_inputs[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', lines.eng[seq_index: seq_index + 1])
    print('Decoded sentence:', decoded_sentence)

NameError: name 'encoder_inputs' is not defined

In [11]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1


NameError: name 'generate_batch' is not defined

In [12]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])


print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

NameError: name 'k' is not defined

In [13]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

NameError: name 'k' is not defined

In [14]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

NameError: name 'k' is not defined

In [15]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

NameError: name 'k' is not defined

In [16]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

NameError: name 'k' is not defined

In [66]:
def translate(sentence):
#     sentence = (decoded_sentence)
    print('Input: %s' % (sentence))
    print('Predicted translation: {}',(decoded_sentence))

In [68]:
translate('this is book. ')

Input: this is book. 
Predicted translation: {}  अध्यापिका देरसे चुना खु़फ़िया निगरानी पूजा पूजा प्रायौगिक
